# Necessary Packages

In [1]:
# Data-handeling/mathmatic packages
import numpy as np
import pandas as pd
import operator


# Plotting packages
import matplotlib.pyplot as plt
import seaborn as sns

#Spotipy utilized packages
import os
import sys
import spotipy
sp = spotipy.Spotify()
from spotipy.oauth2 import SpotifyClientCredentials

# Machine learning packages
# from sklearn import preprocessing, cross_validation, datasets
from sklearn import preprocessing,  datasets
from sklearn.model_selection import cross_validate as cross_validation
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegressionCV
# from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Data Collection

In [2]:
def spotify(cid, secret, username, playlistid):
    
        """This function takes the user's information
        and retrieves the song attributes that are
        attached to the songs in the playlist
        and then creates a global pandas dataframe
        of the song attributes"""

        try:
                client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
                sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
                sp.trace=False                
                
                #global top 50 playlist
                playlist= sp.user_playlist(username,playlistid)
                songs = playlist["tracks"]["items"]
                ids = []
                
                for i in range(len(songs)):
                        ids.append(songs[i]["track"]["id"])
                features = sp.audio_features(ids)
                
                
                #my top 50 playlist
                playlist2 = sp.user_playlist(username,playlistid2)
                songs2 = playlist2['tracks']['items']
                ids2 = []
                
                for j in range(len(songs2)):
                    ids2.append(songs2[j]['track']['id'])
                features2 = sp.audio_features(ids2)
                
                global top_50, my_100
                top_50= pd.DataFrame(features)
                my_100 = pd.DataFrame(features2)
                
                
           
        except:
                print('Error in input varibles')
                sys.exit()


if __name__ == '__main__':
        cid =""
        secret = ""
        username = '21btaxthmh7p3mzr2pto5itja'
        playlistid = ''
        playlistid2 = ''
        spotify(cid, secret, username, playlistid)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Error in input varibles
Traceback (most recent call last):
  File "<ipython-input-2-82924511511b>", line 10, in spotify
    client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
  File "/opt/anaconda/envs/MLCourse/lib/python3.7/site-packages/spotipy/oauth2.py", line 157, in __init__
    self.client_id = client_id
  File "/opt/anaconda/envs/MLCourse/lib/python3.7/site-packages/spotipy/oauth2.py", line 109, in client_id
    self._client_id = _ensure_value(val, "client_id")
  File "/opt/anaconda/envs/MLCourse/lib/python3.7/site-packages/spotipy/oauth2.py", line 77, in _ensure_value
    raise SpotifyOauthError(msg)
spotipy.oauth2.SpotifyOauthError: No client_id. Pass it or set a SPOTIPY_CLIENT_ID environment variable.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda/envs/MLCourse/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(cod

TypeError: object of type 'NoneType' has no len()

# Data manipulation/description

In [ ]:
# Top 50 global songs
top_50.head()

In [ ]:
# My top 100 played songs in 2017
my_100.head()

In [ ]:
# removing unwanted features
top_50 = top_50.drop(['id','analysis_url','track_href','type','uri'],axis =1)
my_100 = my_100.drop(['id','analysis_url','track_href','type','uri'],axis =1)

In [ ]:
# creating classes 
top_50['class'] = ["Global"] * len(top_50)
my_100['class'] = ["My"] * len(my_100)

In [ ]:
# description of each dataset
top_50.describe()

In [ ]:
my_100.describe()

In [ ]:
# combining dataframes
all_data = top_50.append(my_100)

In [ ]:
all_data.describe()

In [ ]:
#converting milliseconds to minutes
all_data['duration_ms'] = (all_data['duration_ms']*0.001)/60
my_100['duration_ms'] = (my_100['duration_ms']*0.001)/60
top_50['duration_ms'] = (top_50['duration_ms']*0.001)/60

#renaming column
all_data = all_data.rename(columns = {'duration_ms' : 'duration_min'})
my_100 = my_100.rename(columns = {'duration_ms' : 'duration_min'})
top_50 = top_50.rename(columns = {'duration_ms' : 'duration_min'})

In [ ]:
sns.pairplot(data = all_data, hue = 'class')
plt.show()

# Machine learning 

In [ ]:
# creating training and test datasets
X = all_data.drop('class',axis =1)
y = all_data['class']

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y, test_size =0.2)

## Decision Tree


In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)

In [ ]:
prediction = dt.predict(X_test)

In [ ]:
# confusion matrix shape is actual x, where prediction position 1 = 'Global' position 2 = 'My'
# i.e. 18 is the correct number of 'My' class predictions
print('Decision Tree Prediction Results')
print(confusion_matrix(y_test,prediction))
print("\n")
print(classification_report(y_test,prediction))

# Random Forest



In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)

In [ ]:
rf_pred = rf.predict(X_test)

In [ ]:
print('Random Forest Prediction Results')
print(confusion_matrix(y_test,rf_pred))
print("\n")
print(classification_report(y_test,rf_pred))

# Feature analysis with Random Forest Model

In [ ]:
rf_feature_importance = rf.feature_importances_
feat_imp_dic
print("Total Influence:",sum(feat_imp_dic.values()))

In [ ]:
top_5_feats = dict(sorted(feat_imp_dic.items(),key = operator.itemgetter(1), reverse = True)[0:5])
top_5_feats

In [ ]:
plt.figure(figsize=(20, 10))
plt.title("Random Forest Feature Importance")
plt.bar(range(len(top_5_feats)), sorted(list(top_5_feats.values()), reverse = True), align='center')
plt.xticks(range(len(top_5_feats)), list(top_5_feats.keys()))
plt.ylabel('Percentage influence')
plt.savefig("Random Forest Feature Importance.png")
plt.show()

In [ ]:
# looking at the relationship between the top 5 influencial features
sns.pairplot(data = all_data, vars = top_5_feats, hue = 'class')
plt.savefig("Feature relation.png")
plt.show()

In [ ]:
print("Top Global 50 Feature Description")
top_50[list(top_5_feats)].describe()

In [ ]:
print("My Top 100 Feature Description")
my_100[list(top_5_feats)].describe()

In [ ]:
# plotting histograms of the top 5 most influencial features
for i in range(len(list(feat_of_interest.keys()))):
    plt.figure(figsize=(16, 16))
    plt.subplot(3, 2, i+1)
    plt.title('Distribution of %s' % feat_of_interest.keys()[i])
    plt.hist(feat_of_interest.values[:,i], bins = 'auto')
    plt.xlabel('Feature Value')
    plt.ylabel('Frequency')
    plt.savefig("Feature Frequency distribution.png")
    plt.show()

In [ ]:
print("Correlation of top features")
correlation = feat_of_interest.corr()
correlation

In [ ]:
# Creating a subset from the top 5 influencial features
subset = all_data[['danceability', 'duration_min', 'loudness', 'speechiness', 'liveness', 'class']]

In [ ]:
#Converting classes to binary (Global = 1, My = 0)
subset['class'] = subset['class'].str.contains('Global').astype(int)

In [ ]:
x = subset[['danceability','speechiness']].as_matrix()
X = preprocessing.scale(x)
y = subset['class'].as_matrix()

In [ ]:
#logistic regression
lr = LogisticRegressionCV()
lr.fit(X,y)

In [ ]:
# Generate a dataset and plot it
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)
plt.scatter(X[:,0], X[:,1], s=40, c=y, cmap=plt.cm.Spectral)
plt.title('Divergent of Classes')
plt.xlabel('Danceability')
plt.ylabel('Speechiness')

In [ ]:
def plot_decision_boundary(pred_func):
    # Set min and max values and give it some padding
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole gid
    Z = pred_func(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.scatter(X[:, 0], X[:, 1],
                edgecolor = 'black',linewidth = '1',s=50, c=y, cmap=plt.cm.Spectral)

In [ ]:
plot_decision_boundary(lambda x: lr.predict(x))
plt.title("Decision Boundary Logistic Regression")
plt.xlabel('Danceability')
plt.ylabel('Speechiness')
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y, test_size =50)


In [ ]:
#visializing how accurate the decision boundary is


plot_decision_boundary(lambda x: lr.predict(x))
plt.title("Decision Boundary with Predictions")
plt.xlabel('Danceability')
plt.ylabel('Speechiness')
plt.scatter(X_test[:,0], X_test[:,1], marker = "^",alpha = 0.78,edgecolor = 'green',linewidth = '2', s=800, c=y_test, cmap=plt.cm.Spectral)
plt.show()

# Neural Network

In [ ]:
num_examples = len(X_train) # training set size
nn_input_dim = X_train.shape[1] # input layer dimensionality
nn_output_dim = 2 # output layer dimensionality

# learning rate for gradient descent (typically set to 0.01)
lr = 0.01 

In [ ]:
def neural_network_training(inputs,desired,nn_hdim,epochs):
    
    
    np.random.seed(3)
    w1 = 2 * np.random.random((nn_input_dim,nn_hdim))-1
    w2 = 2 * np.random.random((nn_hdim,nn_output_dim)) -1
    
    weights = {}
    
    for i in range(epochs):
        
        #forward propagation
        h1 = np.dot(inputs,w1)
        a1 = np.tanh(h1)
        h2 = np.dot(a1,w2)
        exp_h2 = np.exp(h2)
        probability = exp_h2 / np.sum(exp_h2, axis = 1, keepdims=True)

        #backward propagation
        delta2 = probability
        delta2[range(num_examples),desired] -=1
        dw2 = a1.T.dot(delta2)
        delta1 = delta2.dot(w2.T) * (1 - np.power(a1,2))
        dw1 = np.dot(inputs.T,delta1)

        w1 += -lr * dw1
        w2 += -lr * dw2


        weights = {'w1': w1, 'w2':w2}

    return weights

In [ ]:
def prediction(nn, inputs,desired):
    
    w1, w2 = nn['w1'], nn['w2']
    
    #forward prop
    h1 = np.dot(inputs,w1)
    a1 = np.tanh(h1)
    h2 = np.dot(a1,w2)
    exp_h2 = np.exp(h2)
    probability = exp_h2 / np.sum(exp_h2, axis = 1, keepdims=True)
    return sum(np.argmax(probability,axis = 1) == desired)/num_examples


In [ ]:
print("Results of Backward Propagating Neural Network")
hid_dim = [1,2,3,4,5,6]
for i, nn_hdim in enumerate(hid_dim):
    nn = neural_network_training(X_train,y_train,nn_hdim,1000)
    print("training prediction accuracy with %i hidden layer node(s): %s" %(i,str(prediction(nn,X_train,y_train))))
print('\n')
for i, nn_hdim in enumerate(hid_dim):
    nn = neural_network_training(X_train,y_train,nn_hdim,1000)
    print("test prediction accuracy with %i hidden layer node(s): %s" %(i,str(prediction(nn,X_test,y_test))))

In [ ]:
def plot_prediction(nn, inputs):
    
    w1, w2 = nn['w1'], nn['w2']
    
    #forward prop
    h1 = np.dot(inputs,w1)
    a1 = np.tanh(h1)
    h2 = np.dot(a1,w2)
    exp_h2 = np.exp(h2)
    probability = exp_h2 / np.sum(exp_h2, axis = 1, keepdims=True)
    return np.argmax(probability,axis = 1) 

In [ ]:
model = neural_network_training(X_train,y_train,15,1000)

plot_decision_boundary(lambda x: plot_prediction(model, x))
plt.title("Decision Boundary for hidden layer size 15")
plt.xlabel('Danceability')
plt.ylabel('Speechiness')
plt.savefig("Decision Boundary for hidden layer size 15.png")

In [ ]:
# plotting the test accuracy
plt.figure(figsize=(16, 32))
hid_dim = [1,2,3,4,5,50]
for i, nn_hdim in enumerate(hid_dim):
    plt.subplot(5, 2, i+1)
    plt.title('Hidden Layer size %d' % nn_hdim)
    plt.xlabel('Danceability')
    plt.ylabel('Speechiness')
    model = neural_network_training(X_train,y_train,nn_hdim,1000)
    plot_decision_boundary(lambda x: plot_prediction(model, x))
    plt.savefig("Decision Boundary for various hidden layer sizes.png")
plt.show()

In [ ]:
model = neural_network_training(X_train,y_train,2,1000)

plot_decision_boundary(lambda x: plot_prediction(model, x))
plt.scatter(X_test[:,0], X_test[:,1], marker = "^",alpha = 0.78,edgecolor = 'green',linewidth = '2', s=800, c=y_test, cmap=plt.cm.Spectral)
plt.title("Decision Boundary for hidden layer size 2")
plt.xlabel('Danceability')
plt.ylabel('Speechiness')

plt.savefig("Decision Boundary for hidden layer size 2.png")